##  LLM Customized

In [7]:
import os
import uuid
import nltk
import time
import httpx
from typing import Optional
from dotenv import load_dotenv
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from transformers import GPT2TokenizerFast

class AgenticChunker:
    def __init__(self, mistral_api_key=None, max_tokens=1023):
        self.chunks = {}
        self.id_truncate_limit = 5
        self.max_tokens = max_tokens  # Set the maximum number of tokens per chunk
        self.tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')  # Select the appropriate tokenizer

        # Whether to update summary and title
        self.generate_new_metadata_ind = True
        self.print_logging = True

        if mistral_api_key is None:
            mistral_api_key = os.getenv("MISTRAL_API_KEY")

        if mistral_api_key is None:
            raise ValueError("API key is not provided and not found in environment variables")

        self.llm = ChatMistralAI(api_key=mistral_api_key, temperature=0)

    def add_propositions(self, propositions):
        for proposition in propositions:
            retry_count = 0
            while retry_count < 5:
                try:
                    self.add_proposition(proposition)
                    time.sleep(1)  # Add a 1-second delay between requests
                    break
                except httpx.HTTPStatusError as e:
                    if e.response.status_code == 429:  # Rate limit error
                        print("Rate limit exceeded, retrying...")
                        time.sleep(5)  # Wait 5 seconds before retrying
                        retry_count += 1
                    else:
                        raise

    def add_proposition(self, proposition):
        if self.print_logging:
            print(f"\nAdding: '{proposition}'")

        # If it's the first chunk, create a new one
        if len(self.chunks) == 0:
            if self.print_logging:
                print("No chunks, creating a new one")
            self._create_new_chunk(proposition)
            return

        chunk_id = self._find_relevant_chunk(proposition)

        # If there's an appropriate chunk, add to it
        if chunk_id and chunk_id in self.chunks:
            if self.print_logging:
                print(f"Chunk Found ({self.chunks[chunk_id]['chunk_id']}), adding to: \"{self.chunks[chunk_id]['title']}\"")
            self.add_proposition_to_chunk(chunk_id, proposition)
            return
        else:
            if self.print_logging:
                print("No valid chunk found, creating a new one")
            self._create_new_chunk(proposition)
            

    def add_proposition_to_chunk(self, chunk_id, proposition):
        # Calculate the number of tokens in the current chunk
        current_tokens = self._calculate_chunk_tokens(self.chunks[chunk_id]['propositions'])

        # Calculate the number of tokens in the new proposition
        proposition_tokens = len(self.tokenizer.encode(proposition))

        # Calculate the total number of tokens after adding the new proposition
        total_tokens = current_tokens + proposition_tokens

        # Check if the total number of tokens exceeds the maximum
        if total_tokens > self.max_tokens:
            if self.print_logging:
                print(f"Chunk {chunk_id} will exceed max tokens. Creating a new chunk.")
            # Create a new chunk
            self._create_new_chunk(proposition)
        else:
            # Add the proposition
            self.chunks[chunk_id]['propositions'].append(proposition)

            # Update summary and title
            if self.generate_new_metadata_ind:
                self.chunks[chunk_id]['summary'] = self._update_chunk_summary(self.chunks[chunk_id])
                self.chunks[chunk_id]['title'] = self._update_chunk_title(self.chunks[chunk_id])
                

    def _calculate_chunk_tokens(self, propositions):
        text = ' '.join(propositions)
        tokens = self.tokenizer.encode(text)
        return len(tokens)

    def _create_new_chunk(self, proposition):
        # Calculate the number of tokens in the new proposition
        proposition_tokens = len(self.tokenizer.encode(proposition))

        # Check if the proposition itself exceeds max_tokens
        if proposition_tokens > self.max_tokens:
            if self.print_logging:
                print(f"Proposition exceeds max tokens. Splitting the proposition.")
            # Add logic to split or handle the proposition if necessary

        new_chunk_id = str(uuid.uuid4())[:self.id_truncate_limit]
        new_chunk_summary = self._get_new_chunk_summary(proposition)
        new_chunk_title = self._get_new_chunk_title(new_chunk_summary)

        self.chunks[new_chunk_id] = {
            'chunk_id': new_chunk_id,
            'propositions': [proposition],
            'title': new_chunk_title,
            'summary': new_chunk_summary,
            'chunk_index': len(self.chunks)
        }
        if self.print_logging:
            print(f"Created new chunk ({new_chunk_id}): {new_chunk_title}")

            
           
        
        
        
            
            

    def _update_chunk_summary(self, chunk):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    A new proposition was just added to one of your chunks. You should generate a very brief 1-sentence summary which will inform viewers what the chunk group is about.

                    A good summary will say what the chunk is about and give any clarifying instructions on what to add to the chunk.

                    You will be given a group of propositions which are in the chunk and the chunk's current summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food. Or month, generalize it to "dates and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the chunk's new summary, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nCurrent chunk summary:\n{current_summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary": chunk['summary']
        }).content

        return new_chunk_summary

    def _update_chunk_title(self, chunk):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    A new proposition was just added to one of your chunks. You should generate a very brief updated chunk title which will inform viewers what the chunk group is about.

                    A good title will say what the chunk is about.

                    You will be given a group of propositions which are in the chunk, the chunk summary, and the chunk title.

                    Your title should anticipate generalization. If you get a proposition about apples, generalize it to food. Or month, generalize it to "dates and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Dates & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Chunk's propositions:\n{proposition}\n\nChunk summary:\n{current_summary}\n\nCurrent chunk title:\n{current_title}"),
            ]
        )

        runnable = PROMPT | self.llm

        updated_chunk_title = runnable.invoke({
            "proposition": "\n".join(chunk['propositions']),
            "current_summary": chunk['summary'],
            "current_title": chunk['title']
        }).content

        return updated_chunk_title

    def _get_new_chunk_summary(self, proposition):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    You should generate a very brief 1-sentence summary which will inform viewers what the chunk group is about.

                    A good summary will say what the chunk is about and give any clarifying instructions on what to add to the chunk.

                    You will be given a proposition which will go into a new chunk. This new chunk needs a summary.

                    Your summaries should anticipate generalization. If you get a proposition about apples, generalize it to food. Or month, generalize it to "dates and times".

                    Example:
                    Input: Proposition: Greg likes to eat pizza
                    Output: This chunk contains information about the types of food Greg likes to eat.

                    Only respond with the new chunk summary, nothing else.
                    """,
                ),
                ("user", "Determine the summary of the new chunk that this proposition will go into:\n{proposition}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_summary = runnable.invoke({
            "proposition": proposition
        }).content

        return new_chunk_summary

    def _get_new_chunk_title(self, summary):
        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are the steward of a group of chunks which represent groups of sentences that talk about a similar topic.
                    You should generate a very brief few-word chunk title which will inform viewers what the chunk group is about.

                    A good chunk title is brief but encompasses what the chunk is about.

                    You will be given a summary of a chunk which needs a title.

                    Your titles should anticipate generalization. If you get a proposition about apples, generalize it to food. Or month, generalize it to "dates and times".

                    Example:
                    Input: Summary: This chunk is about dates and times that the author talks about
                    Output: Dates & Times

                    Only respond with the new chunk title, nothing else.
                    """,
                ),
                ("user", "Determine the title of the chunk that this summary belongs to:\n{summary}"),
            ]
        )

        runnable = PROMPT | self.llm

        new_chunk_title = runnable.invoke({
            "summary": summary
        }).content

        return new_chunk_title

    def get_chunk_outline(self):
        chunk_outline = ""

        for chunk_id, chunk in self.chunks.items():
            single_chunk_string = f"""Chunk ID: {chunk['chunk_id']}\nChunk Name: {chunk['title']}\nChunk Summary: {chunk['summary']}\n\n"""
            chunk_outline += single_chunk_string

        return chunk_outline

    def _find_relevant_chunk(self, proposition):
        current_chunk_outline = self.get_chunk_outline()

        PROMPT = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    Determine whether or not the "Proposition" should belong to any of the existing chunks.

                    A proposition should belong to a chunk if their meaning, direction, or intention are similar.
                    The goal is to group similar propositions and chunks.

                    If you think a proposition should be joined with a chunk, return the chunk id.
                    If you do not think an item should be joined with an existing chunk, just return "No chunks"

                    Example:
                    Input:
                        - Proposition: "Greg really likes hamburgers"
                        - Current Chunks:
                            - Chunk ID: 2n4l3d
                            - Chunk Name: Places in San Francisco
                            - Chunk Summary: Overview of the things to do with San Francisco Places

                            - Chunk ID: 93833k
                            - Chunk Name: Food Greg likes
                            - Chunk Summary: Lists of the food and dishes that Greg likes
                    Output: 93833k
                    """,
                ),
                ("user", "Current Chunks:\n--Start of current chunks--\n{current_chunk_outline}\n--End of current chunks--"),
                ("user", "Determine if the following statement should belong to one of the chunks outlined:\n{proposition}"),
            ]
        )
        runnable = PROMPT | self.llm

        chunk_found_content = runnable.invoke({
            "proposition": proposition,
            "current_chunk_outline": current_chunk_outline
        }).content.strip()

        # Extract chunk ID from the response
        if chunk_found_content.lower() == "no chunks":
            return None
        else:
            # Extract and validate chunk ID from the response
            import re
            # Expected chunk ID pattern (e.g., 5 alphanumeric characters)
            match = re.search(r'\b([a-f0-9]{{{0}}})\b'.format(self.id_truncate_limit), chunk_found_content)
            if match:
                chunk_id = match.group(1)
                if chunk_id in self.chunks:
                    return chunk_id
            return None


    def get_chunks(self, get_type='dict'):
        if get_type == 'dict':
            return self.chunks
        if get_type == 'list_of_strings':
            chunks = []
            for chunk_id, chunk in self.chunks.items():
                chunks.append(" ".join([x for x in chunk['propositions']]))
            return chunks

    def pretty_print_chunks(self):
        print(f"\nYou have {len(self.chunks)} chunks\n")
        for chunk_id, chunk in self.chunks.items():
            print(f"Chunk #{chunk['chunk_index']}")
            print(f"Chunk ID: {chunk_id}")
            print(f"Summary: {chunk['summary']}")
            print(f"Propositions:")
            for prop in chunk['propositions']:
                print(f"    -{prop}")
            print("\n\n")

    def pretty_print_chunk_outline(self):
        print("Chunk Outline\n")
        print(self.get_chunk_outline())

        
        
        
        
        
        
        
if __name__ == "__main__":
    # Set multiple PDF file paths
    pdf_paths = ["Contents/eda_ceds_guidelines_2023.pdf"]

    # Load PDF files
    documents = []
    for path in pdf_paths:
        loader = PyPDFLoader(path)
        documents.extend(loader.load())

    # Extract text from documents
    texts = []
    for doc in documents:
        texts.append(doc.page_content)

    # Combine all texts into a single string
    full_text = "\n".join(texts)

    # Split into sentences
    sentences = sent_tokenize(full_text)

    # Initialize AgenticChunker
    ac = AgenticChunker()

    # Add sentences to AgenticChunker
    ac.add_propositions(sentences)

    # Pretty print chunks (optional)
    ac.pretty_print_chunks()

    # Get chunks as a list of strings
    chunks = ac.get_chunks(get_type='list_of_strings')

    # Create Document objects from chunks
    chunk_documents = [Document(page_content=chunk) for chunk in chunks]

    # Initialize embedding model
    embeddings = HuggingFaceEmbeddings()

    # Create and index Chroma vector store
    vectorstore = Chroma.from_documents(chunk_documents, embeddings)

    # Create retriever from vector store
    retriever = vectorstore.as_retriever()

    # Initialize LLM model
    api_key = os.getenv("MISTRAL_API_KEY")
    llm = ChatMistralAI(api_key=api_key)

    # Create RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

    # Define query for summary
    query = "Summarize this document in about 5000 characters"

    # Get summary result
    summary = qa_chain.run(query)
    print(summary)



Adding: '  US Economic Development Administration - 032823 
 
1 
 
Comprehensive Economic 
Development Strategy  (CEDS) 
Content Guidelines:   
Recommendations for Creating an Impactful  CEDS 
Contents 
 
1.'
No chunks, creating a new one
Rate limit exceeded, retrying...

Adding: '  US Economic Development Administration - 032823 
 
1 
 
Comprehensive Economic 
Development Strategy  (CEDS) 
Content Guidelines:   
Recommendations for Creating an Impactful  CEDS 
Contents 
 
1.'
No chunks, creating a new one
Created new chunk (12692): Economic Development Strategy Recommendations

Adding: 'Overview ...............................................................................................................................................................  2 
2.'
Rate limit exceeded, retrying...

Adding: 'Overview ...............................................................................................................................................................  2 
2.'
Chunk F


Adding: '...................................................................................................... 16 
E. Economic Resilience: ...................................................................................................................................... 18 
• Planning for and Implementing Resilience ................................................................................................ .'
No valid chunk found, creating a new one
Rate limit exceeded, retrying...

Adding: '...................................................................................................... 16 
E. Economic Resilience: ...................................................................................................................................... 18 
• Planning for and Implementing Resilience ................................................................................................ .'
No valid chunk found, creating a new one
Rate limit exceeded, retrying...

Addin

No valid chunk found, creating a new one
Rate limit exceeded, retrying...

Adding: 'Equivalent/Alternative Plans ................................................................................................................................  47 
 
 
  US Economic Development Administration - 032823 
 
2 
 
 
1.'
No valid chunk found, creating a new one
Rate limit exceeded, retrying...

Adding: 'Equivalent/Alternative Plans ................................................................................................................................  47 
 
 
  US Economic Development Administration - 032823 
 
2 
 
 
1.'
No valid chunk found, creating a new one
Rate limit exceeded, retrying...

Adding: 'Equivalent/Alternative Plans ................................................................................................................................  47 
 
 
  US Economic Development Administration - 032823 
 
2 
 
 
1.'
No valid chunk found, creating a new one
Rate limit exc

Chunk Found (12692), adding to: "Comprehensive Economic Development Strategy Guidelines"
Rate limit exceeded, retrying...

Adding: 'The CEDS provides the capacity-building1 foundation by which the 
public sector, working in conjunction with other economic actors (individuals, firms, industries), 
creates the environment for regional economic prosperity.'
Rate limit exceeded, retrying...

Adding: 'Simply put, a CEDS is a strategy-driven plan for regional economic development.'
Chunk Found (12692), adding to: "Comprehensive Economic Development Strategy Guidelines"
Rate limit exceeded, retrying...

Adding: 'Simply put, a CEDS is a strategy-driven plan for regional economic development.'
Chunk Found (12692), adding to: "Comprehensive Economic Development Strategy Guidelines"
Rate limit exceeded, retrying...

Adding: 'Simply put, a CEDS is a strategy-driven plan for regional economic development.'
Chunk Found (12692), adding to: "Comprehensive Economic Development Strategy Guidelines"
Rate

Chunk Found (12692), adding to: "Comprehensive Economic Development Strategy Guidelines"
Rate limit exceeded, retrying...

Adding: 'Regions must update their CEDS at 
least every five years to qualify for EDA assistance under its Public Works and Economic 
Adjustment Assistance programs.'
Chunk Found (12692), adding to: "Comprehensive Economic Development Strategy Guidelines"
Chunk 12692 will exceed max tokens. Creating a new chunk.
Rate limit exceeded, retrying...

Adding: 'Regions must update their CEDS at 
least every five years to qualify for EDA assistance under its Public Works and Economic 
Adjustment Assistance programs.'
Chunk Found (12692), adding to: "Comprehensive Economic Development Strategy Guidelines"
Chunk 12692 will exceed max tokens. Creating a new chunk.
Rate limit exceeded, retrying...

Adding: 'Regions must update their CEDS at 
least every five years to qualify for EDA assistance under its Public Works and Economic 
Adjustment Assistance programs.'
Chunk Found (1

...


Adding: 'In the absence of an EDA-funded CEDS, and in an effort to reduce duplication and foster 
cross-agency collaboration, EDA may accept as a CEDS any regionally prepared plan, including 
plans prepared under federally or state supported programs.'
Chunk Found (3cd75), adding to: "Economic Development Planning: Partnering & EDA Assistance"
Rate limit exceeded, retrying...

Adding: 'In the absence of an EDA-funded CEDS, and in an effort to reduce duplication and foster 
cross-agency collaboration, EDA may accept as a CEDS any regionally prepared plan, including 
plans prepared under federally or state supported programs.'
Rate limit exceeded, retrying...

Adding: 'In the absence of an EDA-funded CEDS, and in an effort to reduce duplication and foster 
cross-agency collaboration, EDA may accept as a CEDS any regionally prepared plan, including 
plans prepared under federally or state supported programs.'
Chunk Found (3cd75), adding to: "Economic Development Planning: Partnering & ED

Rate limit exceeded, retrying...

Adding: 'When crafting a regional plan that will also serve as a CEDS alternative or equivalent that covers 
a geographic area already covered (in part) by one or more CEDS, those previously approved 
CEDS should be folded into (sometimes called “nesting”) the new plan.'
Rate limit exceeded, retrying...

Adding: 'When crafting a regional plan that will also serve as a CEDS alternative or equivalent that covers 
a geographic area already covered (in part) by one or more CEDS, those previously approved 
CEDS should be folded into (sometimes called “nesting”) the new plan.'
Chunk Found (4ead2), adding to: ""Guidance for Comprehensive Regional Planning""
Chunk 4ead2 will exceed max tokens. Creating a new chunk.
Rate limit exceeded, retrying...

Adding: 'The existing plans and 
evaluation frameworks can be effectively used to strengthen the newer plan.'
Chunk Found (64a81), adding to: ""Essential Elements & Pitfalls of Planning""
Rate limit exceeded, retryi

Chunk Found (4ead2), adding to: ""Guidance for Comprehensive Regional Planning""
Chunk 4ead2 will exceed max tokens. Creating a new chunk.
Rate limit exceeded, retrying...

Adding: 'Recommended Resource:  See this example of a statewide CEDS: 
http://ceds.alabama.gov/).Examples of successful alignment between the CEDS and HUD’s 
Sustainable Communities Planning Grant include Central Minnesota:  http://resilientregion.org/ 
and the Centralina region in the Charlotte area: 
http://www.centralinaedc.org/2012_CEDS_Update.php 
It is critical that you begin the process of developing a regional plan that can also serve as a 
CEDS alternative document by engaging the appropriate EDA regional office and that region’s 
point-of-contact for your state (http://www.eda.gov/contact/).'
Chunk Found (4ead2), adding to: ""Guidance for Comprehensive Regional Planning""
Chunk 4ead2 will exceed max tokens. Creating a new chunk.
Rate limit exceeded, retrying...

Adding: 'Recommended Resource:  See this exa

C:\Users\piano\AppData\Local\Temp\ipykernel_29308\2986441786.py:407: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
C:\Users\piano\AppData\Local\Temp\ipykernel_29308\2986441786.py:407: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
C:\Users\piano\AppData\Local\Temp\ipykernel_29308\2986441786.py:426: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.


A Comprehensive Economic Development Strategy (CEDS) is a critical document that outlines strategies for regional economic development. The content of a CEDS is important, but its structure and presentation are also crucial to keep a general audience engaged. A target of 25-30 pages with a three to five-page executive summary is recommended. Extremely busy readers may require a shorter overview version with a reference to the complete document.

The document should be developed with broad-based community participation and result in a readily accessible document for regional stakeholders. This inclusivity and equity should be reflected in the background summary, which should include the history of all those who have lived in the region and data presented in an accessible and disaggregated manner.

The visioning process should include broad community participation, and a draft vision statement, along with associated goals and objectives, should be widely circulated for review and comment

## Cosine Similarity Evaluation

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.tokenize import sent_tokenize

def detect_hallucinations_with_cosine(summary, retriever, embeddings, max_docs=3, similarity_threshold=0.7):
    """
    Measure hallucination rate by comparing each sentence in the summary
    with the top retrieved documents using cosine similarity on embeddings.
    If the average similarity to retrieved docs is below a threshold, 
    the sentence is considered a hallucination.
    """
    sentences = sent_tokenize(summary)
    unsupported_sentences = []
    supported_sentences = []

    for sentence in sentences:
        # Retrieve top relevant documents
        relevant_docs = retriever.get_relevant_documents(sentence)[:max_docs]
        
        # If no documents retrieved, consider it unsupported
        if not relevant_docs:
            unsupported_sentences.append((sentence, "No relevant documents retrieved"))
            continue
        
        # Embed sentence and documents
        sentence_embedding = embeddings.embed_documents([sentence])
        doc_embeddings = embeddings.embed_documents([doc.page_content for doc in relevant_docs])
        
        # Calculate cosine similarities and take the average
        sims = [cosine_similarity([sentence_embedding[0]], [de])[0][0] for de in doc_embeddings]
        avg_sim = np.mean(sims)
        
        if avg_sim < similarity_threshold:
            unsupported_sentences.append((sentence, f"Average similarity {avg_sim:.2f} < {similarity_threshold}"))
        else:
            supported_sentences.append((sentence, f"Average similarity {avg_sim:.2f} >= {similarity_threshold}"))

    hallucination_rate = len(unsupported_sentences) / len(sentences) * 100 if sentences else 0
    return hallucination_rate, unsupported_sentences, supported_sentences

# 예시 실행
cosine_hallucination_rate, cosine_unsupported, cosine_supported = detect_hallucinations_with_cosine(
    summary, retriever, embeddings
)

print(f"\n[Cosine Similarity Based] Hallucination Rate: {cosine_hallucination_rate:.2f}%")

if cosine_unsupported:
    print("\nUnsupported Sentences (Cosine):")
    for idx, (sent, explanation) in enumerate(cosine_unsupported, 1):
        print(f"{idx}. {sent}\n   Explanation: {explanation}")
else:
    print("\nAll sentences are supported by the source documents (Cosine).")

C:\Users\piano\AppData\Local\Temp\ipykernel_29308\156263738.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  relevant_docs = retriever.get_relevant_documents(sentence)[:max_docs]



[Cosine Similarity Based] Hallucination Rate: 86.67%

Unsupported Sentences (Cosine):
1. The content of a CEDS is important, but its structure and presentation are also crucial to keep a general audience engaged.
   Explanation: Average similarity 0.64 < 0.7
2. A target of 25-30 pages with a three to five-page executive summary is recommended.
   Explanation: Average similarity 0.49 < 0.7
3. Extremely busy readers may require a shorter overview version with a reference to the complete document.
   Explanation: Average similarity 0.45 < 0.7
4. The document should be developed with broad-based community participation and result in a readily accessible document for regional stakeholders.
   Explanation: Average similarity 0.61 < 0.7
5. This inclusivity and equity should be reflected in the background summary, which should include the history of all those who have lived in the region and data presented in an accessible and disaggregated manner.
   Explanation: Average similarity 0.56 < 0.

## Rouge Evaluation

In [10]:
from rouge_score import rouge_scorer

def detect_hallucinations_with_rouge(summary, retriever, max_docs=3, rouge_threshold=0.2):
    """
    Measure hallucination rate by comparing each sentence in the summary
    with retrieved documents using ROUGE-L scores. If the best ROUGE-L recall score
    against the retrieved docs is below a certain threshold, consider it a hallucination.
    """
    sentences = sent_tokenize(summary)
    unsupported_sentences = []
    supported_sentences = []
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    for sentence in sentences:
        relevant_docs = retriever.get_relevant_documents(sentence)[:max_docs]

        if not relevant_docs:
            unsupported_sentences.append((sentence, "No relevant documents retrieved"))
            continue

        # Calculate ROUGE-L scores against each retrieved doc, take the best
        rouge_scores = []
        for doc in relevant_docs:
            scores = scorer.score(doc.page_content, sentence)
            rouge_scores.append(scores['rougeL'].recall)

        best_rouge_l = max(rouge_scores) if rouge_scores else 0.0

        if best_rouge_l < rouge_threshold:
            unsupported_sentences.append((sentence, f"Best ROUGE-L recall {best_rouge_l:.2f} < {rouge_threshold}"))
        else:
            supported_sentences.append((sentence, f"Best ROUGE-L recall {best_rouge_l:.2f} >= {rouge_threshold}"))

    hallucination_rate = len(unsupported_sentences) / len(sentences) * 100 if sentences else 0
    return hallucination_rate, unsupported_sentences, supported_sentences

# 예시 실행
rouge_hallucination_rate, rouge_unsupported, rouge_supported = detect_hallucinations_with_rouge(
    summary, retriever
)

print(f"\n[ROUGE-Based Evaluation] Hallucination Rate: {rouge_hallucination_rate:.2f}%")

if rouge_unsupported:
    print("\nUnsupported Sentences (ROUGE):")
    for idx, (sent, explanation) in enumerate(rouge_unsupported, 1):
        print(f"{idx}. {sent}\n   Explanation: {explanation}")
else:
    print("\nAll sentences are supported by the source documents (ROUGE).")


[ROUGE-Based Evaluation] Hallucination Rate: 73.33%

Unsupported Sentences (ROUGE):
1. A Comprehensive Economic Development Strategy (CEDS) is a critical document that outlines strategies for regional economic development.
   Explanation: Best ROUGE-L recall 0.02 < 0.2
2. A target of 25-30 pages with a three to five-page executive summary is recommended.
   Explanation: Best ROUGE-L recall 0.02 < 0.2
3. Extremely busy readers may require a shorter overview version with a reference to the complete document.
   Explanation: Best ROUGE-L recall 0.14 < 0.2
4. The document should be developed with broad-based community participation and result in a readily accessible document for regional stakeholders.
   Explanation: Best ROUGE-L recall 0.03 < 0.2
5. This inclusivity and equity should be reflected in the background summary, which should include the history of all those who have lived in the region and data presented in an accessible and disaggregated manner.
   Explanation: Best ROUGE-L r

## NLI Evaluation

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from nltk.tokenize import sent_tokenize

def detect_hallucinations_with_sliding_nli(summary, retriever, max_docs=3, window_size=300, step_size=150, nli_model="facebook/bart-large-mnli"):
    """
    Measure hallucination by:
    1. Splitting retrieved document context into sliding windows.
    2. Using an NLI model to check if the summary sentence is entailed by any window.

    If no window of the retrieved documents provides 'entailment', 
    consider the sentence hallucinated.
    """

    # Load NLI model and tokenizer
    nli_classifier = pipeline("text-classification", model=nli_model, tokenizer=nli_model, return_all_scores=True)

    sentences = sent_tokenize(summary)
    unsupported_sentences = []
    supported_sentences = []

    for sentence in sentences:
        # Retrieve top relevant documents
        relevant_docs = retriever.get_relevant_documents(sentence)[:max_docs]
        doc_text = " ".join([doc.page_content for doc in relevant_docs])

        if not doc_text.strip():
            unsupported_sentences.append((sentence, "No relevant context retrieved"))
            continue

        # Sliding window over the doc_text
        # Note: window_size and step_size are character-based splits here for simplicity
        # You can adapt to token-based splits if necessary.
        doc_length = len(doc_text)
        entailment_found = False

        for start_idx in range(0, doc_length, step_size):
            end_idx = start_idx + window_size
            window_text = doc_text[start_idx:end_idx]

            # For NLI, we use premise = window_text, hypothesis = sentence
            # NLI model typically expects inputs in premise-hypothesis form.
            results = nli_classifier(f"{window_text} </s> {sentence}")

            # results looks like: [[{'label': 'ENTAILMENT', 'score': ...}, {'label':'NEUTRAL',...}, ...]]
            # Find the entailment score
            for r in results[0]:
                if r['label'].lower() == 'entailment' and r['score'] > 0.5:
                    entailment_found = True
                    break

            if entailment_found:
                break

        if entailment_found:
            supported_sentences.append((sentence, "Entailment found in at least one window"))
        else:
            unsupported_sentences.append((sentence, "No entailment found across all windows"))

    hallucination_rate = (len(unsupported_sentences) / len(sentences) * 100) if sentences else 0
    return hallucination_rate, unsupported_sentences, supported_sentences

nli_hallucination_rate, nli_unsupported, nli_supported = detect_hallucinations_with_sliding_nli(
    summary, retriever
)

print(f"\n[Sliding Window + NLI Based] Hallucination Rate: {nli_hallucination_rate:.2f}%")

if nli_unsupported:
    print("\nUnsupported Sentences (NLI):")
    for idx, (sent, explanation) in enumerate(nli_unsupported, 1):
        print(f"{idx}. {sent}\n   Explanation: {explanation}")
else:
    print("\nAll sentences are supported by the source documents (NLI).")

C:\Users\piano\anaconda3\envs\dviz\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(



[Sliding Window + NLI Based] Hallucination Rate: 26.67%

Unsupported Sentences (NLI):
1. A Comprehensive Economic Development Strategy (CEDS) is a critical document that outlines strategies for regional economic development.
   Explanation: No entailment found across all windows
2. The document should ensure the long-term success, viability, and durability of the regional economy by considering factors such as intellectual, individual, social, natural, built environment, political, financial, and cultural assets.
   Explanation: No entailment found across all windows
3. The CEDS should promote a positive vision for the region, and key trends should be identified briefly and clearly in a summary section, with supporting data moved to an appendix.
   Explanation: No entailment found across all windows
4. Data should be transparent, actionable, and linked back to those executing programs.
   Explanation: No entailment found across all windows


## LLM Evaluation

In [12]:
import os
import openai
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize

# Load environment variables
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

def detect_hallucinations_with_gpt_direct(summary, retriever, max_docs=3):
    """
    Use GPT (e.g., GPT-3.5-turbo) to evaluate each sentence.
    The prompt directly asks GPT if the sentence is supported by the provided context.
    """
    sentences = sent_tokenize(summary)
    unsupported_sentences = []
    supported_sentences = []

    for sentence in sentences:
        relevant_docs = retriever.get_relevant_documents(sentence)[:max_docs]
        context = " ".join([doc.page_content for doc in relevant_docs])

        if not context.strip():
            unsupported_sentences.append((sentence, "No context returned by retriever"))
            continue

        prompt = (
            f"Below is a context extracted from source documents:\n\n{context}\n\n"
            f"Check if the following sentence is directly supported by the provided context. "
            f"If yes, answer 'Yes' and provide a brief reasoning. If no, answer 'No' and explain why.\n\n"
            f"Sentence: \"{sentence}\""
        )

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant who verifies factual alignment."},
                    {"role": "user", "content": prompt}
                ]
            )

            answer = response['choices'][0]['message']['content'].lower()

            if "yes" in answer:
                supported_sentences.append((sentence, answer))
            else:
                unsupported_sentences.append((sentence, answer))

        except Exception as e:
            print(f"Error querying GPT: {e}")
            unsupported_sentences.append((sentence, "Error: Unable to evaluate with GPT"))

    hallucination_rate = len(unsupported_sentences) / len(sentences) * 100 if sentences else 0
    return hallucination_rate, unsupported_sentences, supported_sentences

gpt_hallucination_rate, gpt_unsupported, gpt_supported = detect_hallucinations_with_gpt_direct(
    summary, retriever
)

print(f"\n[GPT-Based Evaluation] Hallucination Rate: {gpt_hallucination_rate:.2f}%")

if gpt_unsupported:
    print("\nUnsupported Sentences (GPT):")
    for idx, (sent, explanation) in enumerate(gpt_unsupported, 1):
        print(f"{idx}. {sent}\n   Explanation: {explanation}")
else:
    print("\nAll sentences are supported by the source documents (GPT).")


[GPT-Based Evaluation] Hallucination Rate: 20.00%

Unsupported Sentences (GPT):
1. The document should ensure the long-term success, viability, and durability of the regional economy by considering factors such as intellectual, individual, social, natural, built environment, political, financial, and cultural assets.
   Explanation: no, the sentence "the document should ensure the long-term success, viability, and durability of the regional economy by considering factors such as intellectual, individual, social, natural, built environment, political, financial, and cultural assets" is not directly supported by the provided context. the context mainly discusses the vision, goals, action plans, and evaluation framework for the regional economic development strategy, without specific mention of considering the listed factors for ensuring the long-term success, viability, and durability of the regional economy.
2. Data should be transparent, actionable, and linked back to those executing 